In [1]:
import pandas as pd
import requests
from datetime import datetime
from plyer import notification

In [14]:
def extrair_cep(cep):
    url = f"https://brasilapi.com.br/api/cep/v1/{cep}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame.from_dict(data, orient='index').T
        return df
        raise Exception(f"Erro na requisição: {response.status_code} - {url}\n{response.text}")


In [ ]:
def get_base_ceps(ceps):
    df_ceps_full = pd.DataFrame()

    for cep in ceps:
        try:
            df_cep = extrair_cep(cep)
            
            df_ceps_full = pd.concat([df_ceps_full, df_cep], ignore_index=True)
        
        except Exception as e:
            print(f"Erro ao consultar o CEP {cep}: {e}")

    return df_ceps_full


In [17]:
ceps = ["01001000", "01002000", "99999999"]  
try:
    base_ceps = get_base_ceps(ceps)
    print("Consulta realizada com sucesso:")
    print(base_ceps)
except Exception as e:
    print(f"Erro geral: {e}")


Consulta realizada com sucesso:
        cep state       city          neighborhood              street  \
0  01001000    SP  São Paulo                    Sé         Praça da Sé   
1  01002000    SP  São Paulo                    Sé         Rua Direita   
2  99999999    PR    Sarandi  Jardim Centro Cívico  Avenida das Torres   

    service  
0  open-cep  
1  open-cep  
2  open-cep  


In [18]:
cep = "40330120"

try:
    resultado = extrair_cep(cep)
    print("Consulta realizada com sucesso:")
    print(resultado)
except Exception as e:
    print(f"Erro: {e}")


Consulta realizada com sucesso:
        cep state      city neighborhood           street   service
0  40330120    BA  Salvador         IAPI  Avenida Galícia  open-cep


In [22]:
def extrair_cep(cep):
    url = f"https://brasilapi.com.br/api/cep/v1/{cep}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame.from_dict(data, orient='index').T
        return df
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            errors = response.json().get("errors", [])
            for error in errors:
                if error["message"].startswith("CEP INVÁLIDO"):
                    alerta(1, "Consulta CEP", erro="CEP inválido fornecido pelo usuário.")
                elif error["message"].startswith("CEP não encontrado"):
                    alerta(2, "Consulta CEP", erro="CEP válido, mas não encontrado em serviços externos.")
        elif response.status_code == 500:
            alerta(3, "Consulta CEP", erro="Erro interno no serviço de terceiros.")
        else:
            alerta(3, "Consulta CEP", erro=f"Erro HTTP inesperado: {e}")
        return None
    except requests.exceptions.RequestException as e:
        alerta(3, "Consulta CEP", erro=f"Erro de conexão: {e}")
        return None